In [1]:

import os
import csv
from dotenv import load_dotenv
import google.generativeai as genai

In [2]:
load_dotenv(dotenv_path='C:/Users/pooja konduri/OneDrive/Desktop/BridgeLabs/GenAI_tasks/key.env')
genai.configure(api_key=os.getenv('GEMINI_API_KEY'))

In [3]:
model = genai.GenerativeModel(
    model_name="gemini-1.5-flash"
)

In [4]:
def read_emails(file_path,delimiter="--- End Email ---"):
    """Reads emails from a file."""
    with open('C:/Users/pooja konduri/OneDrive/Desktop/BridgeLabs/GenAI_tasks/input/emails.txt', 'r') as file:
        emails = file.read().split(delimiter)
    return [email.strip() for email in emails if email.strip()]

In [6]:
def extract_email_info(email):
    """Extracts sender, receiver, and body from an email."""
    lines = email.split("\n")
    sender = receiver = body = subject = ""
    
    for line in lines:
        if line.startswith("From:"):
            sender = line.split(":")[1].strip()
        elif line.startswith("To:"):
            receiver = line.split(":")[1].strip()
        elif line.startswith("Subject:"):
            subject = line.split(":")[1].strip()
        elif not line.startswith("From:") and not line.startswith("To:") and not line.startswith("Subject:"):
            body += line.strip() + " "
    
    return sender, receiver, subject, body.strip()

In [7]:
def summarize_email(body, chat_session):
    """Summarizes the email body using Gemini AI."""
    response = chat_session.send_message(f"Summarize this email: {body}")
    return response.text

In [8]:
def translate_email(text, chat_session):
    """Translates text to Hindi using Gemini AI."""
    
    response = chat_session.send_message(f"Translate the following text to Hindi: {text}")
    return response.text

In [9]:
def save_to_csv(data, csv_file):
    """Saves data to a CSV file."""
    with open(csv_file, 'w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerow(["Sender", "Receiver", "Subject", "Summary (EN)", "Summary (HI)"])
        writer.writerows(data)

In [10]:
def process_emails(file_path, csv_file, chat_session, delimiter="--- End Email ---"):
    """
    Processes the emails by extracting the information, summarizing, and translating them.
    
    Args:
    - file_path: The path to the input file containing emails.
    - csv_file: The path to the output CSV file.
    - chat_session: The Gemini AI chat session.
    """
    emails = read_emails(file_path, delimiter)
    data = []

    for email in emails:
        sender, receiver, subject, body = extract_email_info(email)
        summary_en = summarize_email(body, chat_session)
        summary_hi = translate_email(summary_en, chat_session)
        data.append([sender, receiver, subject, summary_en, summary_hi])
    
    save_to_csv(data, csv_file)

In [11]:
def main():
    # Start the chat session with the model
    chat_session = model.start_chat(history=[])

    # Process emails and save results to CSV
    process_emails('emails.txt', 'processed_emails.csv', chat_session)

if __name__ == '__main__':
    main()